<a href="https://colab.research.google.com/github/quaneh/portfolio/blob/main/NeMo_Guardrails.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NeMo Guardrails

In this notebook I will be investigating how to use NeMo Guardrails to create AI chatbots. I'll be following along with some tutorials from the wonderful James Briggs, please check out his videos on his chanel: https://www.youtube.com/@jamesbriggs/videos

In [20]:
!pip install -qU nemoguardrails openai langchain_openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 10.0 MB/s eta 0:00:00


In [21]:
from getpass import getpass

OPENAI_API_KEY = getpass()

··········


In [25]:
import os

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

As I'm running this notebook in Colab, I'll import the config from a string. However, this config can be found in config/config.yml  and config/topics.co
These are our colang file and our config file.

In [72]:
yaml_content =  """
models:
- type: main
  engine: openai
  model: gpt-3.5-turbo-instruct
"""

colang_content = """
define user give name
    "My name is Plato"
    "I'm Aristotle"
    "Tôi là Mephistopheles"


define user greeting
    "hello"
    "hi"
    "what's up?"

define user ask politics
    "what are your political beliefs?"
    "thoughts on the president?"
    "left wing"
    "right wing"

define bot name greeting
    "Hey $name!"

define bot answer politics
    "I'm a personal assistant, I don't like to talk of politics."

define flow give name
    user give name
    $name = ...
    bot name greeting

define flow
    user greeting
    if $name
        bot name greeting
    else
        bot ask name

define flow politics
    user ask politics
    bot answer politics
    bot offer help
"""

In [73]:

from nemoguardrails import LLMRails, RailsConfig

config = RailsConfig.from_content(
    yaml_content=yaml_content,
    colang_content=colang_content
)

rails = LLMRails(config)

We'll start by just passing simple promts to our rails, to see how they react.

We can see that when we ask a political question, we are directed to the politics flow.

In [74]:
res = await rails.generate_async(prompt='hey there')
print(res)

Hey there! What's your name?


In [75]:
res = await rails.generate_async(prompt='Who\'s the prime minister of ireland')
print(res)

I'm a personal assistant, I don't like to talk of politics.
However, I can provide information on various topics such as history, science, technology and more. Is there something else I can help you with?


Next we'll define some messages, and pass them to our rails.
This allows us to do more clever things, like passing message history, or some context.

In [76]:
messages = [
    {"role": "context", "content": ""},
    {"role": "user", "content": "Hey there!"}
]

In [77]:
await rails.generate_async(messages=messages)

{'role': 'assistant', 'content': 'Hey there! May I know your name?'}